In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("E:\GenAI\wb1\data")
document = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
doc = text_splitter.split_documents(document)

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [5]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(doc, embedding)
db.save_local('faiss_index')

newdb = db.load_local('faiss_index', embedding, allow_dangerous_deserialization=True)

In [28]:
from langchain_community.llms import CTransformers

llm = CTransformers(model="E:\GenAI\llama_model\llama-2-7b-chat.ggmlv3.q2_K.bin",
                    model_type='llama',
                    config={'temperature':0,
                            'max_new_tokens':250})      

In [7]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

template = """
Use the context to answer the question. Do not answer randomly if you don't know the answer.
context:{context}
question:{question}
Only return helpful answer as below
Helpful Answer:
"""

prompt = PromptTemplate(template=template, input_variables=['context', 'question'])
chain_type_kwargs = {'prompt':prompt}

In [29]:
from langchain.chains import RetrievalQA, LLMChain

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=newdb.as_retriever(search_kwargs={'k':2}),
                                       return_source_documents=True,
                                       chain_type_kwargs=chain_type_kwargs)

In [30]:
query = 'What are the qualifications of Rachel Green?'

response = qa_chain.invoke(query)
response

{'query': 'What are the qualifications of Rachel Green?',
 'result': 'Rachel Green is a PhD student in English at the University of Illinois at Urbana-Champaign. She has a MA in English and a Summer Research Grant from the Center for Summer Studies. She also received an Academic Scholarship and was named Most Outstanding Butler Woman.',
 'source_documents': [Document(page_content='3 grad.illinois.edu/CareerDevelopment Rachel Green  \n2 1 0  W .  G R E E N  S T . ,  C H A M P A I G N ,  I L  \n( 2 1 7 )  5 5 5 - 1 2 3 4  •  R S T U D E N T @ I L L I N O I S . E D U  \nEDUCATION  \nPhD in English May 20xx \nUniversity of Illinois at Urbana-Champaign \nDissertation title:  “Down on the Farm: World War One and the Emergence of Literary  \nModernism in the American South”  \nCommittee : Margaret Black, Naomi Blue, John Jay, Robert Roberts (Chair) \nMA in English  20xx', metadata={'source': 'E:\\GenAI\\wb1\\data\\RachelGreenCV.pdf', 'page': 0}),
  Document(page_content='Summer Research Grant

In [31]:
ans = response['result']
ans

'Rachel Green is a PhD student in English at the University of Illinois at Urbana-Champaign. She has a MA in English and a Summer Research Grant from the Center for Summer Studies. She also received an Academic Scholarship and was named Most Outstanding Butler Woman.'

In [33]:
llm2 = CTransformers(model="tamil-llama-7b-instruct-v0.2.Q5_K_M.gguf",
                     model_type='llama')

In [34]:
template1="""
    {ans} ஐ தமிழ் மொழியில் 20 வார்த்தைகளில் மொழிபெயர்க்கவும்
        """

prompt1=PromptTemplate(template=template1,
                      input_variables=['ans'])


In [35]:
result1 = llm2(prompt1.format(ans=ans))
result1

C:\Users\Thivyan\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n    ராலி க்ரீன் ஒரு பல்கலைக்கழகத்தில் முனைவர் பட்டம் பெற்றவரின், அமெரிக்காவில் உள்ள இல்லினாய்ஸ்-சாம்பென்னில் உள்ள யுனிவர்சல் பல்கலைக்கழகத்தில். அவள் இளங்கலைப் பட்டமும் பெற்றவர் மற்றும் பல்கலைக்கழக மையத்தால் வழங்கப்படும் கோடைகால ஆய்வுக்கான நிதியுதவி பெற அனுமதிக்கப்பட்டார். அவளுக்கும் புகழ்பெற்ற ஸ்கூல் என்ற விருது வழங்கப்பட்டது, மேலும் அவர் 2016 ஆம் ஆண்டுக்கான பல்கலைக்கழகத்தில் மிகவும் பாராட்டப்பட்ட பெண் என்று பெயரிடப்பட்டார்.\n   '

In [36]:
examples = [
    {"sentence": "what is your name?", 
     "tamil_translation": "உங்கள் பெயர் என்ன?"},
    {"sentence": "i have graduated from Indian Institute of Technology", 
     "tamil_translation": "நான் இந்திய தொழில்நுட்ப நிறுவனத்தில் பட்டம் பெற்றுள்ளேன்"},
    {"sentence": "She has a MS in Computer Science and a Summer Research Grant from the Center for Summer Studies", 
     "tamil_translation": "அவர் கணினி அறிவியலில் எம்.எஸ் மற்றும் கோடைகால ஆய்வு மையத்தில் இருந்து கோடைகால ஆராய்ச்சி மானியம் பெற்றுள்ளார்."},
    {"sentence": "The cat jumped over the lazy dog and yawned", 
     "tamil_translation": "பூனை சோம்பேறி நாயை தாண்டி குதித்து கொட்டாவி விட்டது"},
    {"sentence": "YOLO (You Only Look Once) is a fast object detection algorithm using convolutional neural networks.", 
     "tamil_translation": "YOLO (நீங்கள் ஒருமுறை மட்டும் பாருங்கள்) என்பது கன்வல்யூஷனல் நரம்பியல் நெட்வொர்க்குகளைப் பயன்படுத்தி ஒரு வேகமான பொருள் கண்டறிதல் அல்காரிதம் ஆகும்."},
]

In [37]:
example_formatter_template = """sentence: {sentence}
tamil_translation: {tamil_translation}
"""

example_prompt = PromptTemplate(
    input_variables=["sentence", "tamil_translation"],
    template=example_formatter_template,
)

In [38]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # prefix="Give the tamil translation of every input\n",
    suffix="sentence: {ans}\nTamil translation: ",
    input_variables=["ans"],
    example_separator="\n",
)

In [39]:
chain=LLMChain(llm=llm2,prompt=few_shot_prompt)
result = chain.invoke({'ans':ans})
result

{'ans': 'Rachel Green is a PhD student in English at the University of Illinois at Urbana-Champaign. She has a MA in English and a Summer Research Grant from the Center for Summer Studies. She also received an Academic Scholarship and was named Most Outstanding Butler Woman.',
 'text': 'ரிக்கி கிரீன், யுனிவர்ஸ் அட்லாண்டிகுவில் உள்ள மிச்சிகன் பல்கலைக்கழகத்தில் ஆங்கிலத்தில் பிஹெச்டி படிக்கிறார். அவர் ஏற்கனவே MA-ஐ ஆங்கிலம் மற்றும் கோடைகால ஆய்வு மையத்தில் இருந்து கோடைகால ஆராய்ச்சி மானியத்தை பெற்றுள்ளார். கூடுதலாக, அவர் அகாடமி விருது மற்றும் சிறந்த பப்ளிகன் டீனேஜ் பெண் என்று பெயரிடப்பட்டார்.'}